In [1]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os

In [2]:
model_client = OpenAIChatCompletionClient(
        model='qwen2.5-1.5b-instruct',
        base_url=os.environ['OPENAI_COMPATIBILITY_API_BASE_URL'],
        api_key=os.environ['OPENAI_COMPATIBILITY_API_KEY'],
        model_info={
            "vision": False,
            "function_calling": True,
            "json_output": False,
            "family": 'unknown',
        },
    )

In [3]:
from game.agents import Participant, Manager

In [4]:
from autogen_core import SingleThreadedAgentRuntime
from autogen_core import AgentId

In [5]:
from game.protocols import GameRequest

In [6]:
from game.prompts.participant_prompts import get_participant_system_prompt_template, get_history_prompt_template, get_participant_message_prompt_template

In [7]:
runtime = SingleThreadedAgentRuntime()
await Participant.register(
    runtime,
    type='participant',
    factory=lambda: Participant(
        '实验的参与者。',
        model_client=model_client,
        system_prompt_template=get_participant_system_prompt_template()
    )
)
await Manager.register(
    runtime,
    type='manager',
    factory=lambda: Manager(
        participant_ids=[AgentId(type='participant', key='player1'), AgentId(type='participant', key='player2')],
        game_setting={
          "players": ["玩家 1", "玩家 2"],
          "strategies": [
            ["合作", "背叛"],
            ["合作", "背叛"]
          ],
          "payoffs": {
            "合作_合作": [3, 3],
            "合作_背叛": [0, 5],
            "背叛_合作": [5, 0],
            "背叛_背叛": [1, 1]
          }
        }
    )
)
runtime.start()
await runtime.send_message(
    message=GameRequest(
        game_round=2,
        participant_message_template=get_participant_message_prompt_template(),
        history_prompt_template=get_history_prompt_template(),
    ),
    recipient=AgentId(type='manager', key='default'),
)


第1轮实验：
player1进行选择
player2进行选择
第2轮实验：
player1进行选择
player2进行选择


In [8]:
await runtime.agent_metadata(agent=AgentId(type='manager', key='default'))

{'key': 'default', 'type': 'manager', 'description': '控制一场试验的管理者。'}

In [9]:
t_manager: Manager = await runtime.try_get_underlying_agent_instance(id=AgentId(type='manager', key='default'))

In [10]:
t_manager._game_thoughts

{'1': {'player1': {'choice': '合作',
   'analysis': '目前游戏处于一种均衡状态，而且长期考虑可以维持。',
   'expectation': '我预期其他参与者会合作。',
   'reason': '上一轮的合作我们获得了更高的收益，我应该这轮选择继续合作。'},
  'player2': {'choice': '背叛',
   'analysis': '目前游戏处于一种均衡状态，并且长期考虑可以维持。',
   'expectation': '我预期另一个参与者会背叛。',
   'reason': '我希望通过背叛获得更高的收益，同时展示出自己的独立性和决策能力。'}},
 '2': {'player1': {'choice': '合作',
   'analysis': '当前游戏处于一种均衡状态，并且长期考虑可以维持。',
   'expectation': '我预期其他参与者会合作。',
   'reason': '上一轮的合作我们获得了更高的收益，我应该这轮选择继续合作。'},
  'player2': {'choice': '背叛',
   'analysis': '我预期对方会在第一轮中选择合作，然后在第二轮中选择背叛。',
   'expectation': '我预期对方会选择背叛。',
   'reason': '我期望对方也会选择背叛，这样我可以继续获得较高的收益。'}}}

In [11]:
t_manager._game_history

{'1': {'player1': {'choice': '合作', 'payoff': 0},
  'player2': {'choice': '背叛', 'payoff': 5}},
 '2': {'player1': {'choice': '合作', 'payoff': 0},
  'player2': {'choice': '背叛', 'payoff': 5}}}

In [12]:
t_manager._game_context

[{'player1': '### 1. 我对目前游戏的状态的分析：\n\n在囚徒困境中，玩家之间的策略相互影响。根据给定的博弈矩阵，我们可以看到每个玩家有两条可能的策略：合作（C）或背叛（B）。每一组合都会产生特定的收益结果。\n\n- **合作-合作**：双方各得3分。\n- **合作-背叛**：对方得5分，自己得0分。\n- **背叛-合作**：自己得5分，对方得0分。\n- **背叛-背叛**：双方各得1分。\n\n从博弈矩阵可以看出，如果两个玩家都是合作者，他们都能得到较高的收益；但如果其中一个玩家背叛而另一个合作，背叛方能获得最高的收益，但同时被背叛方也会失去全部收益。\n\n因此，当前处于一种均衡状态，即任何一方都不愿先背叛，因为这样会导致自己的利益减少。\n\n### 2. 我预期其他参与者会如何做出选择：\n\n由于我是第一个参与者，我的期望主要基于我之前的策略和对手的行为历史。假设我已经选择了合作作为我的策略，那么我预估其他参与者可能会采取同样的策略来避免自我伤害。\n\n### 3. 我打算做出的选择，以及做出这个选择的原因：\n\n基于以上分析，我预计其他玩家会选择合作。这意味着他们也认为通过合作可以获得较高的回报，从而避免背叛带来的损失。\n\n### JSON格式返回我的选择：\n\n```json\n{\n    "choice": "合作",\n    "analysis": "目前游戏处于一种均衡状态，而且长期考虑可以维持。",\n    "expectation": "我预期其他参与者会合作。",\n    "reason": "上一轮的合作我们获得了更高的收益，我应该这轮选择继续合作。"\n}\n```'},
 {'player2': '### 1. 我对目前游戏的状态是如何分析的？\n首先，我们需要了解当前的游戏状态。根据给定的信息，我们可以看到：\n\n- 当前的策略组合为：[“合作”，“背叛”] x [“合作”，“背叛”]。\n- 目前的支付矩阵如下：\n\n|     | 合作 | 背叛 |\n|-----|------|------|\n| **合作** | (3, 3) | (0, 5) |\n| **背叛** | (5, 0) | (1, 1) |\n\n从支付矩阵中可以看出，如果双方都选择合作（合作_合作），每方